In [8]:
import pandas as pd
from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT
from langdetect import detect

def extract_keywords(text):
    # Initialize the vectorizer
    vectorizer = KeyphraseCountVectorizer()

    # Fit and transform the text to get keyphrases
    document_keyphrase_matrix = vectorizer.fit_transform([text]).toarray()
    keyphrases = vectorizer.get_feature_names_out()

    # Initialize the KeyBERT model
    kw_model = KeyBERT()

    # Extract keywords using KeyBERT
    keywords = kw_model.extract_keywords(docs=[text], keyphrase_ngram_range=(1, 2))
    keyword_dict = dict(keywords)
    return keyword_dict

def filter_keywords_by_language(keywords, target_language='en'):
    # Filter keywords based on language
    language_filtered_keywords = {}

    for keyword in keywords:
        if detect(keyword) == target_language:
            language_filtered_keywords[keyword] = keywords[keyword]

    return language_filtered_keywords

def get_ngram_theme(text):
    try:
        # Extract keywords from the text
        extracted_keywords = extract_keywords(text)

        # Filter keywords by language
        filtered_by_language = filter_keywords_by_language(extracted_keywords)

        # Limit to the top 5 unique keywords
        if len(filtered_by_language) >= 5:
            filtered_by_language = dict(list(filtered_by_language.items())[:5])

        return filtered_by_language
    except:
        return {"Null": 0.5}

# Example usage
if __name__ == "__main__":
    text = """
    Supervised learning is the machine learning task of learning a function that
    maps an input to an output based on example input-output pairs. It infers a
    function from labeled training data consisting of a set of training examples.
    In supervised learning, each example is a pair consisting of an input object
    (typically a vector) and a desired output value (also called the supervisory signal).
    A supervised learning algorithm analyzes the training data and produces an inferred function,
    which can be used for mapping new examples. An optimal scenario will allow for the
    algorithm to correctly determine the class labels for unseen instances. This requires
    the learning algorithm to generalize from the training data to unseen situations in a
    'reasonable' way (see inductive bias).
    """

    ngram_theme = get_ngram_theme(text)
    print(ngram_theme)


2023-10-19 10:36:54,151 - KeyphraseVectorizer - INFO - It looks like you do not have downloaded a list of stopwords yet. It is attempted to download the stopwords now.
INFO:KeyphraseVectorizer:It looks like you do not have downloaded a list of stopwords yet. It is attempted to download the stopwords now.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


{'supervised learning': 0.6779, 'labeled training': 0.6013}


In [6]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=a6ff46fe3e1a475e5ca1711efbe915487dac67ec0ca300fa879446014f4cf0c1
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [2]:
!pip install keyphrase_vectorizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.9 MB/s eta 0:00:00


In [4]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.7 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=424621292982da2867452b7d77cf61508e07f0c9613bc684775911b5081c520b
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=b0ee7995853b705448100fa358402b05014a7214fdb1c164f7ffdfd1fc28a8b0
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


In [ ]:
# I separated the keyword extraction, language filtering, and similarity filtering into distinct functions.
# It's important to adjust the similarity_threshold to control the degree of similarity for removing keywords.
# This should make your code more organized and easier to maintain.

In [12]:
from itertools import combinations
import nltk
from nltk.util import ngrams

nltk.download('punkt')  # Download NLTK punkt tokenizer data

def get_ngram_theme(text):
    try:
        # Extract keywords from the text
        extracted_keywords = extract_keywords(text)

        # Filter keywords by language
        filtered_by_language = filter_keywords_by_language(extracted_keywords)

        # Remove duplicate words
        unique_keywords = list(filtered_by_language.keys())

        # Tokenize the text
        words = nltk.word_tokenize(text)

        # Generate Uni, Bi, and Tri-grams
        unigrams = list(ngrams(words, 1))
        bigrams = list(ngrams(words, 2))
        trigrams = list(ngrams(words, 3))

        # Limit to the top 5 unique n-gram combinations for each
        if len(unigrams) >= 5:
            unigrams = unigrams[:5]
        if len(bigrams) >= 5:
            bigrams = bigrams[:5]
        if len(trigrams) >= 5:
            trigrams = trigrams[:5]

        return unique_keywords, unigrams, bigrams, trigrams
    except:
        return {"Null": 0.5}, [], [], []

# Example usage with additional functionality
if __name__ == "__main__":
    text = """
    Supervised learning is the machine learning task of learning a function that
    maps an input to an output based on example input-output pairs. It infers a
    function from labeled training data consisting of a set of training examples.
    In supervised learning, each example is a pair consisting of an input object
    (typically a vector) and a desired output value (also called the supervisory signal).
    A supervised learning algorithm analyzes the training data and produces an inferred function,
    which can be used for mapping new examples. An optimal scenario will allow for the
    algorithm to correctly determine the class labels for unseen instances. This requires
    the learning algorithm to generalize from the training data to unseen situations in a
    'reasonable' way (see inductive bias).
    """

    keywords, unigrams, bigrams, trigrams = get_ngram_theme(text)
    print("Keywords:", keywords)
    print("Unigrams:", unigrams)
    print("Bigrams:", bigrams)
    print("Trigrams:", trigrams)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Keywords: ['supervised learning', 'labeled training']
Unigrams: [('Supervised',), ('learning',), ('is',), ('the',), ('machine',)]
Bigrams: [('Supervised', 'learning'), ('learning', 'is'), ('is', 'the'), ('the', 'machine'), ('machine', 'learning')]
Trigrams: [('Supervised', 'learning', 'is'), ('learning', 'is', 'the'), ('is', 'the', 'machine'), ('the', 'machine', 'learning'), ('machine', 'learning', 'task')]


In [ ]:
# The get_ngram_theme function now returns both the unique keywords (after language filtering and removing duplicates)
#  and the important n-grams (Uni, Bi, Tri) based on the input text.